In [82]:
import pandas as pd
import nltk
import numpy as np
import string
from scipy.sparse import csr_matrix
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.notebook import tqdm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [87]:
pip instanltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:1125)>


False

In [2]:
RANDOM_SEED = 21

In [3]:
df_negative = pd.read_csv('./data/processedNegative.csv').T.reset_index()
df_negative['class'] = 0

In [4]:
df_neutral = pd.read_csv('./data/processedNeutral.csv').T.reset_index()
df_neutral['class'] = 1

In [5]:
df_possitive = pd.read_csv('./data/processedPositive.csv').T.reset_index()
df_possitive['class'] = 2

In [6]:
df = df_negative.append(df_possitive).append(df_neutral)
df.reset_index(drop=True,inplace=True)
df.columns = ['text', 'class']

# Preprocesing

In [7]:
stop_words = nltk.corpus.stopwords.words('english')

In [8]:
text = ' '.join(df.text.map(str.lower).map(str.split).map(lambda x: [i for i in x if i not in stop_words]).map(' '.join).values)

In [9]:
text = text.translate(str.maketrans('', '', string.punctuation))

In [10]:
df.text = df.text\
.map(str.lower) \
.map(lambda x : x.translate(str.maketrans('', '', string.punctuation))) \
.map(str.split) \
.map(lambda x: [i for i in x if i not in stop_words]) \
.map(' '.join)

In [11]:
df.head()

,text,class
0,unhappy dogs like though,0
1,talking driver im goinghe said hed love go new...,0
2,anybody know rands likely fall dollar got mone...,0
3,miss going gigs liverpool unhappy,0
4,isnt new riverdale tonight unhappy,0


In [12]:
index_train, index_test =  map(lambda x: x.index, train_test_split(df.loc[:,'class'],stratify=df.loc[:,'class'], random_state=RANDOM_SEED))

In [13]:
index_train, index_val = map(lambda x: x.index ,train_test_split(df.loc[index_train,'class'], 
                                                                 stratify=df.loc[index_train, 'class'], 
                                                                 test_size=0.2,
                                                                )
                            )

# JUST TOKENIZATIONS

In [14]:
token_df_train = pd.Series(df.loc[index_train,'text']).str.get_dummies(sep=' ')

In [15]:
token_df_train.head()

,0,04,0570,0845am,09,0xx9,0xxxxxx938,1,10,100,...,youve,yoyou,yoyour,yoyoure,yoyoyou,yura,zac,zcc,zero,zoo
745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
token_df_test = pd.Series(df.loc[index_test,'text']).str.get_dummies(sep=' ')

In [17]:
token_df_test = token_df_test.loc[:, set(token_df_train.columns).intersection(set(token_df_test))]

In [18]:
token_df_test.loc[:, set(token_df_train.columns) - set(token_df_train.columns).intersection(set(token_df_test.columns))] = 0

In [19]:
token_df_test

,definitely,roy,tech,feb,sasikala,loads,use,cbi,flash,floodplain,...,ambition,hears,industrialists,ravel,repeals,violentagainst,copy,criticism,proves,bypassing
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## CountVectorizer

In [20]:
count_vectorizer = CountVectorizer()

In [21]:
count_vectorizer.fit(df.loc[index_train,'text'])

CountVectorizer()

In [22]:
count_df_train = pd.DataFrame(data = count_vectorizer.transform(df.loc[index_train,'text']).toarray(),
                              columns=count_vectorizer.get_feature_names(), 
                              index=index_train
                             )

In [23]:
count_df_test = pd.DataFrame(data = count_vectorizer.transform(df.loc[index_test,'text']).toarray(),
                             columns=count_vectorizer.get_feature_names(), 
                             index=index_test,
                            )

In [24]:
count_df_test

,04,0570,0845am,09,0xx9,0xxxxxx938,10,100,1000,100m,...,youve,yoyou,yoyour,yoyoure,yoyoyou,yura,zac,zcc,zero,zoo
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TFIDFVectorizer

In [25]:
tfidf_vectorizer =  TfidfVectorizer()

In [26]:
tfidf_vectorizer.fit(df.loc[index_train, 'text'])

TfidfVectorizer()

In [27]:
tfidf_df_train = pd.DataFrame(data = tfidf_vectorizer.transform(df.loc[index_train,'text']).toarray(),
                              columns=tfidf_vectorizer.get_feature_names(), 
                              index=index_train
                             )

In [28]:
tfidf_df_test = pd.DataFrame(data = tfidf_vectorizer.transform(df.loc[index_test,'text']).toarray(),
                              columns=tfidf_vectorizer.get_feature_names(), 
                              index=index_test
                             )

## Classification

In [29]:
bayes_class = MultinomialNB()
logreg = LogisticRegression() 

In [30]:
index_train, index_val = map(lambda x: x.index ,train_test_split(df.loc[index_train,'class'], 
                                                                 stratify=df.loc[index_train, 'class'], 
                                                                 test_size=0.2,
                                                                )
                            )

### Token

In [31]:
token_df_train.loc[index_train]

,0,04,0570,0845am,09,0xx9,0xxxxxx938,1,10,100,...,youve,yoyou,yoyour,yoyoure,yoyoyou,yura,zac,zcc,zero,zoo
239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1284,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X_train, X_test, y_train, y_test = token_df_train.loc[index_train], token_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.853763440860215

In [33]:
X_train, X_test, y_train, y_test = token_df_train.loc[index_train], token_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8451612903225807

In [34]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(df.loc[index_test, 'class'], logreg.predict(token_df_test))}")
print(f"Байес - {accuracy_score(df.loc[index_test, 'class'], bayes_class.predict(token_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.4055727554179567
Байес - 0.3240454076367389


### Count

In [35]:
X_train, X_test, y_train, y_test = count_df_train.loc[index_train], count_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']

In [36]:
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8473118279569892

In [37]:
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.853763440860215

In [38]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(df.loc[index_test, 'class'], logreg.predict(count_df_test))}")
print(f"Байес - {accuracy_score(df.loc[index_test, 'class'], bayes_class.predict(count_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.8668730650154799
Байес - 0.8441692466460269


### TFIDF

In [39]:
X_train, X_test, y_train, y_test = tfidf_df_train.loc[index_train], tfidf_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']

In [40]:
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.8580645161290322

In [41]:
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8279569892473119

In [42]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(df.loc[index_test, 'class'], logreg.predict(tfidf_df_test))}")
print(f"Байес - {accuracy_score(df.loc[index_test, 'class'], bayes_class.predict(tfidf_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.8648090815273478
Байес - 0.849329205366357


# STEMING

In [43]:
X, y = df['text'].copy(), df['class']

In [44]:
stemmer = PorterStemmer()

In [45]:
X = X.map(lambda x: ' '.join([stemmer.stem(i) for i in x.split()]))

In [46]:
token_df_train = pd.Series(X.loc[index_train]).str.get_dummies(sep=' ')

In [47]:
token_df_train.head()

,0,04,0570,0845am,09,1,10,100,100m,101,...,youu,youuu,youv,yoyou,yoyour,yoyoy,yura,zac,zero,zoo
239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1284,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
token_df_test = pd.Series(X.loc[index_test]).str.get_dummies(sep=' ')

In [49]:
token_df_test = token_df_test.loc[:, set(token_df_train.columns).intersection(set(token_df_test))]

In [50]:
token_df_test.loc[:, set(token_df_train.columns) - set(token_df_train.columns).intersection(set(token_df_test.columns))] = 0

In [51]:
token_df_test

,youll,feb,tech,roy,develop,extrem,trick,quacktopia,oath,lol,...,dodid,media,fame,bare,rs500,gr,bleed,femal,ravel,selfi
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## CountVectorizer

In [52]:
count_vectorizer = CountVectorizer()

In [53]:
count_vectorizer.fit(X.loc[index_train])

CountVectorizer()

In [54]:
count_df_train = pd.DataFrame(data = count_vectorizer.transform(X.loc[index_train]).toarray(),
                              columns=count_vectorizer.get_feature_names(), 
                              index=index_train
                             )

In [55]:
count_df_test = pd.DataFrame(data = count_vectorizer.transform(X.loc[index_test]).toarray(),
                             columns=count_vectorizer.get_feature_names(), 
                             index=index_test,
                            )

In [56]:
count_df_test

,04,0570,0845am,09,10,100,100m,101,103for03,10am,...,youu,youuu,youv,yoyou,yoyour,yoyoy,yura,zac,zero,zoo
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TFIDFVectorizer

In [57]:
tfidf_vectorizer =  TfidfVectorizer()

In [58]:
tfidf_vectorizer.fit(X.loc[index_train])

TfidfVectorizer()

In [59]:
tfidf_df_train = pd.DataFrame(data = tfidf_vectorizer.transform(X.loc[index_train]).toarray(),
                              columns=tfidf_vectorizer.get_feature_names(), 
                              index=index_train
                             )

In [60]:
tfidf_df_test = pd.DataFrame(data = tfidf_vectorizer.transform(X.loc[index_test]).toarray(),
                              columns=tfidf_vectorizer.get_feature_names(), 
                              index=index_test
                             )

In [61]:
tfidf_df_train.head()

,04,0570,0845am,09,10,100,100m,101,103for03,10am,...,youu,youuu,youv,yoyou,yoyour,yoyoy,yura,zac,zero,zoo
239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Classification

In [62]:
bayes_class = MultinomialNB()
logreg = LogisticRegression() 

In [63]:
index_train, index_val = map(lambda x: x.index ,train_test_split(y.loc[index_train], 
                                                                 stratify=y.loc[index_train], 
                                                                 test_size=0.2,
                                                                )
                            )

### Token

In [69]:
token_df_test

,youll,feb,tech,roy,develop,extrem,trick,quacktopia,oath,lol,...,dodid,media,fame,bare,rs500,gr,bleed,femal,ravel,selfi
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
X_train, X_test, y_train, y_test = token_df_train.loc[index_train], token_df_train.loc[index_val], y.loc[index_train], y.loc[index_val]
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.8333333333333334

In [67]:
X_train, X_test, y_train, y_test = token_df_train.loc[index_train], token_df_train.loc[index_val], y.loc[index_train], y.loc[index_val]
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8763440860215054

In [76]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(y.loc[index_test], logreg.predict(token_df_test))}")
print(f"Байес - {accuracy_score(y.loc[index_test], bayes_class.predict(token_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.4004127966976264
Байес - 0.31785345717234265


### Count

In [77]:
X_train, X_test, y_train, y_test = count_df_train.loc[index_train], count_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']

In [78]:
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8844086021505376

In [79]:
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.8333333333333334

In [81]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(y.loc[index_test], logreg.predict(count_df_test))}")
print(f"Байес - {accuracy_score(y.loc[index_test], bayes_class.predict(count_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.8648090815273478
Байес - 0.847265221878225


### TFIDF

In [187]:
X_train, X_test, y_train, y_test = tfidf_df_train.loc[index_train], tfidf_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']

In [188]:
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.8588640275387264

In [189]:
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8623063683304647

In [190]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(df.loc[index_test, 'class'], logreg.predict(tfidf_df_test))}")
print(f"Байес - {accuracy_score(df.loc[index_test, 'class'], bayes_class.predict(tfidf_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.8679050567595459
Байес - 0.8648090815273478


# LEMMATIZATON

In [43]:
X, y = df['text'].copy(), df['class']

In [83]:
lemmer = WordNetLemmatizer()

In [86]:
lemmer.lemmatize('loving')

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/a19279653/nltk_data'
    - '/Users/a19279653/opt/anaconda3/nltk_data'
    - '/Users/a19279653/opt/anaconda3/share/nltk_data'
    - '/Users/a19279653/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [45]:
X = X.map(lambda x: ' '.join([stemmer.stem(i) for i in x.split()]))

In [46]:
token_df_train = pd.Series(X.loc[index_train]).str.get_dummies(sep=' ')

In [47]:
token_df_train.head()

,0,04,0570,0845am,09,1,10,100,100m,101,...,youu,youuu,youv,yoyou,yoyour,yoyoy,yura,zac,zero,zoo
239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1284,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
token_df_test = pd.Series(X.loc[index_test]).str.get_dummies(sep=' ')

In [49]:
token_df_test = token_df_test.loc[:, set(token_df_train.columns).intersection(set(token_df_test))]

In [50]:
token_df_test.loc[:, set(token_df_train.columns) - set(token_df_train.columns).intersection(set(token_df_test.columns))] = 0

In [51]:
token_df_test

,youll,feb,tech,roy,develop,extrem,trick,quacktopia,oath,lol,...,dodid,media,fame,bare,rs500,gr,bleed,femal,ravel,selfi
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## CountVectorizer

In [52]:
count_vectorizer = CountVectorizer()

In [53]:
count_vectorizer.fit(X.loc[index_train])

CountVectorizer()

In [54]:
count_df_train = pd.DataFrame(data = count_vectorizer.transform(X.loc[index_train]).toarray(),
                              columns=count_vectorizer.get_feature_names(), 
                              index=index_train
                             )

In [55]:
count_df_test = pd.DataFrame(data = count_vectorizer.transform(X.loc[index_test]).toarray(),
                             columns=count_vectorizer.get_feature_names(), 
                             index=index_test,
                            )

In [56]:
count_df_test

,04,0570,0845am,09,10,100,100m,101,103for03,10am,...,youu,youuu,youv,yoyou,yoyour,yoyoy,yura,zac,zero,zoo
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TFIDFVectorizer

In [57]:
tfidf_vectorizer =  TfidfVectorizer()

In [58]:
tfidf_vectorizer.fit(X.loc[index_train])

TfidfVectorizer()

In [59]:
tfidf_df_train = pd.DataFrame(data = tfidf_vectorizer.transform(X.loc[index_train]).toarray(),
                              columns=tfidf_vectorizer.get_feature_names(), 
                              index=index_train
                             )

In [60]:
tfidf_df_test = pd.DataFrame(data = tfidf_vectorizer.transform(X.loc[index_test]).toarray(),
                              columns=tfidf_vectorizer.get_feature_names(), 
                              index=index_test
                             )

In [61]:
tfidf_df_train.head()

,04,0570,0845am,09,10,100,100m,101,103for03,10am,...,youu,youuu,youv,yoyou,yoyour,yoyoy,yura,zac,zero,zoo
239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Classification

In [62]:
bayes_class = MultinomialNB()
logreg = LogisticRegression() 

In [63]:
index_train, index_val = map(lambda x: x.index ,train_test_split(y.loc[index_train], 
                                                                 stratify=y.loc[index_train], 
                                                                 test_size=0.2,
                                                                )
                            )

### Token

In [69]:
token_df_test

,youll,feb,tech,roy,develop,extrem,trick,quacktopia,oath,lol,...,dodid,media,fame,bare,rs500,gr,bleed,femal,ravel,selfi
2831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
X_train, X_test, y_train, y_test = token_df_train.loc[index_train], token_df_train.loc[index_val], y.loc[index_train], y.loc[index_val]
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.8333333333333334

In [67]:
X_train, X_test, y_train, y_test = token_df_train.loc[index_train], token_df_train.loc[index_val], y.loc[index_train], y.loc[index_val]
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8763440860215054

In [76]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(y.loc[index_test], logreg.predict(token_df_test))}")
print(f"Байес - {accuracy_score(y.loc[index_test], bayes_class.predict(token_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.4004127966976264
Байес - 0.31785345717234265


### Count

In [77]:
X_train, X_test, y_train, y_test = count_df_train.loc[index_train], count_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']

In [78]:
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8844086021505376

In [79]:
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.8333333333333334

In [81]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(y.loc[index_test], logreg.predict(count_df_test))}")
print(f"Байес - {accuracy_score(y.loc[index_test], bayes_class.predict(count_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.8648090815273478
Байес - 0.847265221878225


### TFIDF

In [187]:
X_train, X_test, y_train, y_test = tfidf_df_train.loc[index_train], tfidf_df_train.loc[index_val], df.loc[index_train, 'class'], df.loc[index_val, 'class']

In [188]:
bayes_class.fit(X_train, y_train)
accuracy_score(y_test, bayes_class.predict(X_test))

0.8588640275387264

In [189]:
logreg.fit(X_train, y_train)
accuracy_score(y_test, logreg.predict(X_test))

0.8623063683304647

In [190]:
print("Результат на тестовом наборе данных:")
print(f"Линейная регрессия - {accuracy_score(df.loc[index_test, 'class'], logreg.predict(tfidf_df_test))}")
print(f"Байес - {accuracy_score(df.loc[index_test, 'class'], bayes_class.predict(tfidf_df_test))}")

Результат на тестовом наборе данных:
Линейная регрессия - 0.8679050567595459
Байес - 0.8648090815273478
